In [1]:
import tensorflow as tf
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from concurrent.futures import ThreadPoolExecutor
from torchvision.utils import make_grid
from PIL import Image
import requests
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from datasets import load_dataset
from torch import nn, Tensor
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import gc
import wandb
from datetime import datetime
import torch
import torch.nn as nn
from torch.cuda.amp import autocast
import os
import multiprocessing

DIM_IN = 2048
DIM_HIDDEN = 4096

class SparseAutoencoder(nn.Module):
    def __init__(self, d_in=DIM_IN, d_hidden=DIM_HIDDEN):
        super().__init__()
        self.encoder = nn.Linear(d_in, d_hidden)
        self.activation = nn.ReLU()
        self.decoder = nn.Linear(d_hidden, d_in)

    def forward(self, x):
        z = self.activation(self.encoder(x))
        x_recon = self.decoder(z)
        return x_recon, z

dtype = torch.float16
if torch.cuda.is_available():
    torch.cuda.set_device(1)  # Set current device to 1
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    dtype = torch.bfloat16

device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

2025-05-02 20:44:06.271919: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-02 20:44:06.296634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746218646.322150 1162385 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746218646.330752 1162385 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746218646.352937 1162385 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Using device: cuda:1


In [2]:
MODEL_ID = "google/paligemma-3b-mix-224"
# TRAINING_DATASET = "imagenet-1k"
# TRAINING_DATASET_SIZE = 5000

model = PaliGemmaForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype
).eval()
model = model.to(device) # Move model to GPU
processor = AutoProcessor.from_pretrained(MODEL_ID)

# Set up activation capture (vision tower)
vision_acts = {}
def vision_hook(module, input, output):
    vision_acts["activation"] = output

hook_handle = model.vision_tower.vision_model.encoder.layers[20].register_forward_hook(vision_hook)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Load in DROID Dataset

In [ ]:
import tensorflow_datasets as tfds
import numpy as np
from PIL import Image
import io
import base64
import html
from IPython.display import HTML, display

# Settings
SEQLEN = 128
NUM_FRAMES = 8  # 🔁 Change this to however many frames you want

# --- Load dataset ---
train_ds = tfds.load("droid_100", split="train[:90%]", data_dir="gs://gresearch/robotics")
val_ds = tfds.load("droid_100", split="train[90%:]", data_dir="gs://gresearch/robotics")

# --- Image preprocessing ---
def preprocess_image(image):
    return image.resize((224, 224))  # for model input

I0000 00:00:1746218740.358063 1162385 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 891 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0
I0000 00:00:1746218740.359922 1162385 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 26284 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:05.0, compute capability: 8.0


: 

Train SAE

In [ ]:
# === Initialize Weights & Biases ===
wandb.init(project="sparse-autoencoder", name="SAE-run", config={
    "hidden_multiplier": 8,
    "learning_rate": 1e-4,
    "sparsity_weight": 1e-2,
    "n_epochs": 50,
    "batch_size": 512,
})

# === Hyperparameters ===
hidden_multiplier = wandb.config.hidden_multiplier
learning_rate = wandb.config.learning_rate
sparsity_weight = wandb.config.sparsity_weight
n_epochs = wandb.config.n_epochs
batch_size = wandb.config.batch_size

# === Prepare SAE ===
d_in = all_activations.shape[-1]
d_hidden = hidden_multiplier * d_in

sae = SparseAutoencoder(d_in=d_in, d_hidden=d_hidden).to(device) 
sae = nn.DataParallel(sae)  # Now wrap for multi-GPU
optimizer = torch.optim.AdamW(sae.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# === Dataset & Loader ===
train_dataset = torch.utils.data.TensorDataset(all_activations)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)